In [19]:
import os
import sys
import json
import re

from glob import glob
from logging import error, warning

from common import CLASS_MAP, load_conllu


CLASS_COMMENT_RE = re.compile(r'^#.*?\bregister:(.*)')


def get_class_from_comments(comments):
    class_ = None
    for comment in comments:
        m = CLASS_COMMENT_RE.match(comment)
        if m:
            if class_:
                raise ValueError('duplicate class')
            class_ = m.group(1)
    return CLASS_MAP[class_]


def load_conllu_with_class(fn):
    sentences, class_ = [], None
    for comments, sentence in load_conllu(fn):
        c = get_class_from_comments(comments)
        if c is not None:
            if class_ is not None:
                raise ValueError('duplicate class')
            class_ = c
        sentences.append(sentence)
    if class_ is None:
        raise ValueError('missing class in {}'.format(fn))
    return sentences, class_


def load_parsed_data(dirpath):
    parses, classes = [], []
    for fn in glob('{}/*.conllu'.format(dirpath)):
        sentences, class_ = load_conllu_with_class(fn)
        parses.append(sentences)
        classes.append(class_)
    return parses, classes


train_parses, train_classes = load_parsed_data('../data/split-parsed/train/')
devel_parses, devel_classes = load_parsed_data('../data/split-parsed/dev/')

KeyError: None

In [5]:
from collections import Counter


MIN_EXAMPLES = 25    # filter classes with fewer


class_count = Counter()
for c in train_classes:
    class_count[c] += 1
target_class = set(c for c, v in class_count.items() if v >= MIN_EXAMPLES)


def filter_by_class(parses, classes, targets):
    filtered_parses, filtered_classes = [], []
    for t, c in zip(parses, classes):
        if c in targets:
            filtered_parses.append(t)
            filtered_classes.append(c)
    return filtered_parses, filtered_classes


train_parses, train_classes = filter_by_class(train_parses, train_classes, target_class)
devel_parses, devel_classes = filter_by_class(devel_parses, devel_classes, target_class)

In [9]:
from pprint import pprint

def class_counts(classes):
    counter = Counter()
    for c in classes:
        counter[c] += 1
    return counter


pprint(class_counts(train_classes))

Counter({'Machine-translated / generated texts': 296,
         'Description with intent to sell': 243,
         'Personal blog': 139,
         'Description of a thing': 103,
         'News reports / news blogs': 93,
         'News reports / News blogs': 91,
         'How-to/instructions': 86,
         'Religious blogs/sermons': 75,
         'Personal opinion blogs': 70,
         'Discussion forums': 65,
         'Reviews': 44,
         'Encyclopedia articles': 35,
         'Community blogs': 33,
         'Sports reports': 30,
         'News+Opinion blogs / Editorials': 26})


In [15]:
def words_from_sentences(sentences):
    words = []
    for sentence in sentences:
        for word in sentence:
            words.append(word.deprel)
    return words


def texts_from_parses(parses):
    texts = []
    for sentences in parses:
        texts.append(' '.join(words_from_sentences(sentences)))
    return texts


train_texts = texts_from_parses(train_parses)
devel_texts = texts_from_parses(devel_parses)

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC


vectorizer = TfidfVectorizer(analyzer='word', lowercase=False, ngram_range=(1,3))
vectorizer.fit(train_texts)

train_X = vectorizer.transform(train_texts)
devel_X = vectorizer.transform(devel_texts)

classifier = LinearSVC(C=1.0)
classifier.fit(train_X, train_classes)

classifier.score(devel_X, devel_classes)

0.5615763546798029

In [17]:
import eli5


eli5.show_weights(classifier, vec=vectorizer, top=(100,100))

In [18]:
import numpy as np

from sklearn.metrics import confusion_matrix
from pandas import DataFrame


pred_Y = classifier.predict(devel_X)
cm = confusion_matrix(devel_classes, pred_Y)
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]    # normalize
df = DataFrame(cm * 100, index=classifier.classes_, columns=classifier.classes_)
df.round(2)

,Community blogs,Description of a thing,Description with intent to sell,Discussion forums,Encyclopedia articles,How-to/instructions,Machine-translated / generated texts,News reports / News blogs,News reports / news blogs,News+Opinion blogs / Editorials,Personal blog,Personal opinion blogs,Religious blogs/sermons,Reviews,Sports reports
Community blogs,0.0,20.00,20.00,0.0,0.0,0.00,0.00,0.00,0.00,0.0,40.00,0.00,0.00,0.00,20.0
Description of a thing,0.0,13.33,53.33,0.0,0.0,6.67,6.67,0.00,13.33,0.0,0.00,0.00,6.67,0.00,0.0
Description with intent to sell,0.0,3.03,75.76,0.0,0.0,6.06,3.03,3.03,3.03,0.0,3.03,0.00,3.03,0.00,0.0
Discussion forums,0.0,0.00,11.11,0.0,0.0,0.00,11.11,0.00,0.00,0.0,66.67,11.11,0.00,0.00,0.0
Encyclopedia articles,0.0,0.00,40.00,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.00,60.00,0.00,0.0
How-to/instructions,0.0,8.33,33.33,0.0,0.0,8.33,16.67,8.33,0.00,0.0,25.00,0.00,0.00,0.00,0.0
Machine-translated / generated texts,0.0,0.00,0.00,0.0,0.0,0.00,100.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.0
News reports / News blogs,0.0,7.69,7.69,0.0,0.0,0.00,7.69,38.46,23.08,0.0,0.00,0.00,15.38,0.00,0.0
News reports / news blogs,0.0,7.69,23.08,0.0,0.0,15.38,7.69,7.69,23.08,0.0,15.38,0.00,0.00,0.00,0.0
News+Opinion blogs / Editorials,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.00,25.00,0.0,25.00,0.00,50.00,0.00,0.0
